In [1]:
# Import Dependencies
import pandas as pd
import numpy as np

In [2]:
# Load and Read File
file = "Resources/purchase_data.csv"
original_df = pd.read_csv(file)
original_df.head()

,Purchase ID,SN,Age,Gender,Item ID,Item Name,Price
0,0,Lisim78,20,Male,108,"Extraction, Quickblade Of Trembling Hands",3.53
1,1,Lisovynya38,40,Male,143,Frenzied Scimitar,1.56
2,2,Ithergue48,24,Male,92,Final Critic,4.88
3,3,Chamassasya86,24,Male,100,Blindscythe,3.27
4,4,Iskosia90,23,Male,131,Fury,1.44


In [3]:
# Determine the number of players
num_players = original_df["SN"].nunique()
pd.DataFrame({"Total Players":[num_players]})

,Total Players
0,576


In [4]:
# Define Total Puchasing Metrics
num_unique_items = original_df["Item ID"].nunique()
average_price = original_df["Price"].mean()
num_of_purchases = original_df["Purchase ID"].nunique()
total_revenue = original_df["Price"].sum()
pd.DataFrame({"Number of Unique Items":[num_unique_items],
             "Average Purchase Price":["$"+str(round(average_price,2))],
             "Number of Purchases":[num_of_purchases],
             "Total Revenue":["$"+'{0:,.2f}'.format(total_revenue)]})

,Number of Unique Items,Average Purchase Price,Number of Purchases,Total Revenue
0,179,$3.05,780,"$2,379.77"


In [5]:
# Remove Duplicates from the Data
original_df_no_duplicates = original_df.drop_duplicates(["SN"])

# Define Gender Demographics
total_genders = pd.DataFrame(original_df_no_duplicates["Gender"].value_counts())
percent_genders = round((total_genders/num_players)*100,2)

# Combine Gender Demographics
summary_genders = pd.concat([total_genders,percent_genders],axis=1)

# Create Gender Demographics Table 
summary_genders.columns = ["Total Number", "Percentage of Players"]

# Print Table
summary_genders

,Total Number,Percentage of Players
Male,484,84.03
Female,81,14.06
Other / Non-Disclosed,11,1.91


In [6]:
# Define Gender Purchase Metrics
purchase_count = original_df.groupby(["Gender"]).count()["Purchase ID"]
avg_purchase_price = round(original_df.groupby(["Gender"]).mean()["Price"],2)
total_purchase_value = round(original_df.groupby(["Gender"]).sum()["Price"],2)
avg_purchase_total_per_person = round(total_purchase_value/original_df_no_duplicates.groupby(["Gender"]).count()["Purchase ID"],2)

# Combine Gender Purchase Metrics
summary_gender_purch_analysis = pd.concat([purchase_count,avg_purchase_price,total_purchase_value,avg_purchase_total_per_person],axis=1)

# Create Gender Purchase Table Columns
summary_gender_purch_analysis.columns = ["Purchase Count","Average Purchase Price","Total Purchase Value","Average Purchase Total per Person by Gender"]

# Format Gender Purchase Table
summary_gender_purch_analysis["Average Purchase Price"] = summary_gender_purch_analysis["Average Purchase Price"].map("${:.2f}".format)
summary_gender_purch_analysis["Total Purchase Value"] = summary_gender_purch_analysis["Total Purchase Value"].map("${:.2f}".format)
summary_gender_purch_analysis["Average Purchase Total per Person by Gender"] = summary_gender_purch_analysis["Average Purchase Total per Person by Gender"].map("${:.2f}".format)

# Print Table
summary_gender_purch_analysis

,Purchase Count,Average Purchase Price,Total Purchase Value,Average Purchase Total per Person by Gender
Gender,,,,
Female,113,$3.20,$361.94,$4.47
Male,652,$3.02,$1967.64,$4.07
Other / Non-Disclosed,15,$3.35,$50.19,$4.56


In [7]:
# Define Age Demographic Bins
age_bins = [0, 9.90, 14.90, 19.90, 24.90, 29.90, 34.90, 39.90, 9999]
bin_names = ["<10", "10-14", "15-19", "20-24", "25-29", "30-34", "35-39", "40+"]

# Separate Players into Age Bins
original_df_no_duplicates["Age Ranges"] = pd.cut(original_df_no_duplicates["Age"],age_bins,labels=bin_names)

# Define Age Demographics
total_age = pd.DataFrame(original_df_no_duplicates["Age Ranges"].value_counts())
percent_age = round((total_age/num_players)*100,2)

# Combine Age Demographics
summary_age = pd.concat([total_age,percent_age],axis=1)

# Creating Table Columns
summary_age.columns = ["Total Number", "Percentage of Players"]

# Print Table Sorted by Age Bins 
summary_age.sort_index()


C:\Users\KCox4\anaconda3\lib\site-packages\ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,Total Number,Percentage of Players
<10,17,2.95
10-14,22,3.82
15-19,107,18.58
20-24,258,44.79
25-29,77,13.37
30-34,52,9.03
35-39,31,5.38
40+,12,2.08


In [8]:
# Define Age Purchase Metrics
original_df["Age Ranges"] = pd.cut(original_df["Age"],age_bins,labels=bin_names)
avg_age = original_df.groupby(["Age Ranges"]).mean()["Price"].rename("Average Purchase Price")
age_purchase_total = original_df.groupby(["Age Ranges"]).sum()["Price"].rename("Total Purchase Value")
age_count = original_df.groupby(["Age Ranges"]).count()["Price"].rename("Purchase Count")
age_purchase_total_per_person = round(age_purchase_total/original_df_no_duplicates.groupby(["Age Ranges"]).count()["Purchase ID"],2)

# Create Age Purchase Table
age_data = pd.DataFrame({"Purchase Count": age_count, "Average Purchase Price": avg_age, "Total Purchase Value": age_purchase_total, "Average Purchase Total Per Person": age_purchase_total_per_person}
    )

# Format Age Purchase Table
age_data["Average Purchase Price"] = age_data["Average Purchase Price"].map("${:,.2f}".format)
age_data["Total Purchase Value"] = age_data["Total Purchase Value"].map("${:,.2f}".format)
age_data ["Purchase Count"] = age_data["Purchase Count"].map("{:,}".format)
age_data["Average Purchase Total Per Person"] = age_data["Average Purchase Total Per Person"].map("${:,.2f}".format)

# Print Table
age_data 

,Purchase Count,Average Purchase Price,Total Purchase Value,Average Purchase Total Per Person
Age Ranges,,,,
<10,23,$3.35,$77.13,$4.54
10-14,28,$2.96,$82.78,$3.76
15-19,136,$3.04,$412.89,$3.86
20-24,365,$3.05,"$1,114.06",$4.32
25-29,101,$2.90,$293.00,$3.81
30-34,73,$2.93,$214.00,$4.12
35-39,41,$3.60,$147.67,$4.76
40+,13,$2.94,$38.24,$3.19


In [9]:
# Calculate Top Spenders
user_total = original_df.groupby(["SN"]).sum()["Price"].rename("Total Purchase Value")
user_average = original_df.groupby(["SN"]).mean()["Price"].rename("Average Purchase Price")
user_count = original_df.groupby(["SN"]).count()["Price"].rename("Purchase Count")

# Create Top Spender Table
user_data = pd.DataFrame({"Purchase Count": user_count,"Average Purchase Price": user_average, "Total Purchase Value": user_total})

# Sort Table by Top 5
summary_top_spenders=user_data.sort_values("Total Purchase Value", ascending=False).head(5)

# Format Table
summary_top_spenders["Average Purchase Price"] = user_data["Average Purchase Price"].map("${:,.2f}".format)
summary_top_spenders["Total Purchase Value"] = user_data["Total Purchase Value"].map("${:,.2f}".format)

# Print Table
summary_top_spenders

,Purchase Count,Average Purchase Price,Total Purchase Value
SN,,,
Lisosia93,5,$3.79,$18.96
Idastidru52,4,$3.86,$15.45
Chamjask73,3,$4.61,$13.83
Iral74,4,$3.40,$13.62
Iskadarya95,3,$4.37,$13.10


In [10]:
# Extract Data from File
item_data = original_df.loc[:,["Item ID", "Item Name", "Price"]]

# Calculate Most Popular Items
total_item_purchase = item_data.groupby(["Item ID","Item Name"]).sum()["Price"].rename("Total Purchase Value")
average_item_purchase = item_data.groupby(["Item ID","Item Name"]).mean()["Price"]
item_count = item_data.groupby(["Item ID","Item Name"]).count()["Price"].rename("Purchase Count")

# Display Table
item_data_pd = pd.DataFrame({"Total Purchase Value": total_item_purchase, "Item Price": average_item_purchase, "Purchase Count": item_count})
item_data_pd = item_data_pd.loc[:,["Purchase Count", "Item Price", "Total Purchase Value"]]
display_popular_item = item_data_pd.sort_values("Purchase Count", ascending=False).head(5)

# Format Table
display_popular_item["Item Price"] = item_data_pd["Item Price"].map("${:,.2f}".format)
display_popular_item["Purchase Count"] = item_data_pd["Purchase Count"].map("{:,}".format)
display_popular_item["Total Purchase Value"] = item_data_pd["Total Purchase Value"].map("${:,.2f}".format)
summary_most_pop_items = display_popular_item.loc[:,["Purchase Count", "Item Price", "Total Purchase Value"]]

# Print Table
summary_most_pop_items

,,Purchase Count,Item Price,Total Purchase Value
Item ID,Item Name,,,
92,Final Critic,13,$4.61,$59.99
178,"Oathbreaker, Last Hope of the Breaking Storm",12,$4.23,$50.76
145,Fiery Glass Crusader,9,$4.58,$41.22
132,Persuasion,9,$3.22,$28.99
108,"Extraction, Quickblade Of Trembling Hands",9,$3.53,$31.77


In [11]:
# Sort Table by Total Purchase Value
display_profitable_item = item_data_pd.sort_values("Total Purchase Value", ascending=False).head(5)

# Format Table
display_profitable_item["Item Price"] = item_data_pd["Item Price"].map("${:,.2f}".format)
display_profitable_item["Purchase Count"] = item_data_pd["Purchase Count"].map("{:,}".format)
display_profitable_item["Total Purchase Value"] = item_data_pd["Total Purchase Value"].map("${:,.2f}".format)
summary_most_profitable_items = display_profitable_item.loc[:,["Purchase Count", "Item Price", "Total Purchase Value"]]

# Print Tables
summary_most_profitable_items

,,Purchase Count,Item Price,Total Purchase Value
Item ID,Item Name,,,
92,Final Critic,13,$4.61,$59.99
178,"Oathbreaker, Last Hope of the Breaking Storm",12,$4.23,$50.76
82,Nirvana,9,$4.90,$44.10
145,Fiery Glass Crusader,9,$4.58,$41.22
103,Singed Scalpel,8,$4.35,$34.80
